"""
We now have two datasets, one is the pygame which matches labels and timestamp,
the other is the openbci EMG data which is 200sr with timestamp. The two programs are launched
almost simultaneously.
We want to use this data to create a training dataset turning the voltage of each electrode
into spectrogram and use CNN to recognize these images. 
"""

In [51]:
import os
import pandas as pd
from datetime import datetime
import scipy.io.wavfile
import numpy as np

pygame_file = "./pygame_data/outputs.csv"
openbci_file = "./openbci_data/data.txt"
new_openbci = "./openbci_data/new_data.txt"
openbci_columns = "id,ch0,ch1,ch2,ch3,accel0,accel1,accel2,timestamp"

In [52]:
temp = open(openbci_file,'r')
data_str = temp.readlines()
temp.close()

cut_data = data_str[6:-1] # delete last row being NA
columns = openbci_columns + "\n"

new = open(new_openbci, "w")
new.writelines(columns)
new.writelines(cut_data)
new.close()

In [53]:
pygame_data = pd.read_csv(pygame_file, sep=",")
openbci_data = pd.read_csv(new_openbci, sep=",")

In [54]:
openbci_data = openbci_data.drop(columns=["accel0","accel1","accel2"])

In [55]:
new_pygame = pygame_data.iloc[5:]
value_to_minus = new_pygame.iloc[0]['timeStamp']
new_pygame['timeStamp'] = new_pygame['timeStamp'] - value_to_minus
new_pygame = new_pygame.drop(columns="keyPressed")


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
def time2millisec(timestamp):
    split_data = timestamp.split(':')
    return (int(split_data[0]) * 60 * 60 + 
            int(split_data[1]) * 60 + 
            float(split_data[2])) * 1000

begin_data = openbci_data['timestamp'][0]
begin_millisec = time2millisec(begin_data)

openbci_data['timestamp'] = openbci_data.timestamp.apply(time2millisec) - begin_millisec

In [64]:
# def series2_2darray(array, sample_rate, dataset_count):
#     delete_nb = dataset_count % sample_rate
#     array = array[:-delete_nb]
#     size = (dataset_count - delete_nb) / sample_rate
#     split_array = np.split(array, size)
#     return np.array(split_array)

ch0 = openbci_data['ch0']
ch1 = openbci_data['ch1']
ch2 = openbci_data['ch2']
ch3 = openbci_data['ch3']

ch0_array = ch0.values
ch1_array = ch1.values
ch2_array = ch2.values
ch3_array = ch3.values

In [68]:
scipy.io.wavfile.write('./openbci_to_wav/ch0.wav', 201, ch0)
scipy.io.wavfile.write('./openbci_to_wav/ch1.wav', 201, ch1)
scipy.io.wavfile.write('./openbci_to_wav/ch2.wav', 201, ch2)
scipy.io.wavfile.write('./openbci_to_wav/ch3.wav', 201, ch3)

# turn the channel data into one wav file according to channel